# Neural Nets v3
`nn_v3`

Should do [Working efficiently with jupyter lab](https://florianwilhelm.info/2018/11/working_efficiently_with_jupyter_lab/)

When this was a notebook with integrated tests, we did: \
`
%load_ext autoreload
%autoreload 2
%matplotlib widget
#%matplotlib inline`

In [ ]:
# import Importing_Notebooks
import numpy as np

A network built of components which:
1. accept an ordered set of reals (we'll use `numpy.array`, and  call them vectors) at the input port and produce another at the output port - this is forward propagation. ${\displaystyle f\colon \mathbf {R} ^{n}\to \mathbf {R} ^{m}}$
1. accept an ordered set of reals at the output port, representing the gradient of the loss function at the output, and produce the gradient of the loss function at the input port - this is back propagation, aka backprop. ${\displaystyle b\colon \mathbf {R} ^{m}\to \mathbf {R} ^{n}}$
1. from the gradient of the loss function at the output, calculate the partial of the loss function w.r.t the internal parameters ${\displaystyle \frac{\partial E}{\partial w} }$
1. accept a scalar $\eta$ to control the adjustment of internal parameters. _Or is this effected by scaling the loss gradient before passing??_
1. update internal parameters ${\displaystyle w \leftarrow w - \eta \frac{\partial E}{\partial w} }$


In [ ]:
class Layer:
    def __init__(self):
        pass
    
    def __call__(self, x):
        """Computes response to input"""
        raise NotImplementedError
        
    def backprop(self, output_delE):
        """Uses output error gradient to adjust internal parameters, and returns gradient of error at input"""
        raise NotImplementedError
        
    def state_vector(self):
        """Provide the layer's learnable state as a vector"""
        raise NotImplementedError

A network built of a cascade of layers:

In [ ]:
class Network:
    def __init__(self):
        self.layers = []
        self.eta = 0.1 #FIXME
        
    def extend(self, net):
        self.layers.append(net)
        
    def __call__(self, input):
        v = input
        for net in self.layers:
            v = net(v)
        return v
    
    def learn(self, facts):
        for (x, expected) in facts:
            y = self(x)
            e = y - expected
            loss = float(e.dot(e.T))/2.0
            egrad = e * self.eta
            for net in reversed(self.layers):
                egrad = net.backprop(egrad)
        return loss

    def state_vector(self):
        """Provide the network's learnable state as a vector"""
        return np.concatenate([layer.state_vector() for layer in self.layers])

___

## Useful Layers

### Identify

In [ ]:
class IdentityLayer(Layer):
    def __call__(self, x):
        return x
    
    def backprop(self, output_delE):
        return output_delE

    def state_vector(self):
        return np.array([])

### Affine
A layer that does an [affine transformation](https://mathworld.wolfram.com/AffineTransformation.html) aka affinity, which is the classic fully-connected layer with output offsets.

$$ \mathbf{M} \mathbf{x} + \mathbf{b} = \mathbf{y} $$
where
$$
\mathbf{x} = \sum_{j=1}^{n} x_j \mathbf{\hat{x}}_j \\
\mathbf{b} = \sum_{i=1}^{m} b_i \mathbf{\hat{y}}_i \\
\mathbf{y} = \sum_{i=1}^{m} y_i \mathbf{\hat{y}}_i
$$
and $\mathbf{M}$ can be written
$$
\begin{bmatrix}
    m_{1,1} & \dots & m_{1,n} \\
    \vdots & \ddots & \vdots \\
    m_{m,1} & \dots & m_{m,n}
\end{bmatrix} \\
$$

#### Error gradient back-propagation
$$ 
\begin{align}
 \frac{\partial loss}{\partial\mathbf{x}}
  &= \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{x}} \\
  &= \mathbf{M}^\mathsf{T}\frac{\partial loss}{\partial\mathbf{y}}
\end{align}
$$

#### Parameter adjustment
$$
 \frac{\partial loss}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{M}}
 = \frac{\partial loss}{\partial\mathbf{y}} \mathbf{x} \\
 \frac{\partial loss}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}} \frac{\partial\mathbf{y}}{\partial\mathbf{b}}
 = \frac{\partial loss}{\partial\mathbf{y}}
$$

#### Adapting to `numpy`

In `numpy` it is more convenient to use row vectors, particularly for calculating the transform on multiple inputs in one operation. We use the identity $ \mathbf{M} \mathbf{x} = (\mathbf{x} \mathbf{M}^\mathsf{T})^\mathsf{T}.$ To avoid cluttering names, we will use `M` in the code below to hold $\mathbf{M}^\mathsf{T}$.

In [ ]:
class AffineLayer(Layer):
    """An affine transformation, which is the classic fully-connected layer with offsets.
    
    The layer has n inputs and m outputs, which numbers must be supplied
    upon creation. The inputs and outputs are marshalled in numpy arrays, 1-D
    in the case of a single calculation, and 2-D when calculating the outputs
    of multiple inputs in one call.
    If called with 1-D array having shape == (n,), e.g numpy.arange(n), it will
    return a 1-D numpy array of shape (m,).
    If called with a 2-D numpy array, input shall have shape (k,n) and will return
    a 2-D numpy array of shape (k,m), suitable as input to a subsequent layer
    that has input width m.
    """
    def __init__(self, n, m):
        self.M = np.empty((n, m))
        self.b = np.empty(m)
        self.randomize()
        
    def randomize(self):
        self.M[:] = np.random.randn(*self.M.shape)
        self.b[:] = np.random.randn(*self.b.shape)
        
    def __call__(self, x):
        self.input = x
        self.output = x @ self.M + self.b
        return self.output
    
    def backprop(self, output_delE):
        last_in = np.atleast_2d(self.input)
        input_delE = output_delE @ self.M.T
        o_delE = np.atleast_2d(output_delE)
        ### FIXME:
        """
        self.M -= np.einsum('ik,jk', output_delE, self.input) \
            if len(output_delE.shape) == 2 \
            else np.outer(self.input, output_delE)
        """
        #t = np.einsum('ki,kj->ji', o_delE, last_in)
        #print(f"M is:\n{self.M}\nt is:\n{t}")
        self.M -= np.einsum('ki,kj->ji', o_delE, last_in)
        self.b -= np.sum(o_delE, 0)       
        return input_delE

    def state_vector(self):
        return np.concatenate((self.M.ravel(), self.b.ravel()))

### Map
Maps a scalar function on the inputs, for e.g. activation layers.

In [ ]:
class MapLayer(Layer):
    """Map a scalar function on the input taken element-wise"""
    def __init__(self, fun, dfundx):
        self.vfun = np.vectorize(fun)
        self.vdfundx = np.vectorize(dfundx)

    def __call__(self, x):
        self.input = x
        return self.vfun(x)
    
    def backprop(self, output_delE):
        input_delE = self.vdfundx(self.input) * output_delE
        return input_delE

    def state_vector(self):
        return np.array([])

---

# Tests
*Dangerously incomplete* \
Mostly `unittest` the `.py` version with a separate test script, see `test-nn_v3.py`.

In [ ]:
if __name__ == '__main__':
    
    two_wide = np.arange(2*4).reshape(-1,2)
    print(f"two_wide is:\n{two_wide}")
    three_wide = np.arange(3*4).reshape(-1,3)
    print(f"three_wide is:\n{three_wide}")
    
    
    # A few very basic tests:
    iL = IdentityLayer()
    assert all(np.equal(iL(np.arange(5)), np.arange(5)))
    assert all(np.equal(iL.backprop(np.arange(7)), np.arange(7)))
    assert np.equal(iL(two_wide), two_wide).all()
    
    mL = MapLayer(lambda x:x**2, lambda d:2*d)
    assert np.equal(mL(np.array([7,3,-11])), np.array([49,9,121])).all()
    assert np.equal(mL.backprop(np.array([2,3,4])), 2 * np.array([2,3,4]) * np.array([7,3,-11])).all()
    assert np.equal(mL(two_wide), np.array([0, 1, 4, 9, 16, 25, 36, 49]).reshape(-1,2)).all()
    #print(f"mL.backprop(np.array([2,3])) is:\n{mL.backprop(np.array([2,3]))}")
    assert np.equal(mL.backprop(np.array([2,3])), 2 * np.array([2,3]) * two_wide).all()

    a = AffineLayer(2,3)
    assert a(np.arange(2)).shape == (3,)
    out = a(np.array([1,-2]))
    assert out.shape == (3,)
    # AffineLayer has parameters that learn
    out_grad = np.array([4, 2, 7]) * 0.001
    in_grad = a.backprop(out_grad)
    print(f"in_grad is:\n{in_grad}")
    print(f"a(two_wide) is:\n{a(two_wide)}")
    bp = a.backprop(three_wide * 0.001)
    print(f"bp is:\n{bp}")

---

To produce an importable `nn_v3.py`:
1. Save this notebook
1. Uncomment the `jupyter nbconvert` line below
1. Execute it.
1. Comment out the convert again
1. Save the notebook again in that form

In [ ]:
###!jupyter nbconvert --to script nn_v3.ipynb